### Pytorch model --> TensorRT examples

#### 1. Load and launch a pretrained model using Pytorch

**Load model**

In [1]:
from torchvision import models
model = models.resnet50(pretrained=True)

c:\Users\ASUS\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [03:07<00:00, 546kB/s]   


**Define preprocessing step**

In [6]:
import cv2
import torch
from albumentations import Resize, Compose
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.transforms import Normalize

In [11]:
def preprocess_image(img_path):
    # Transform for image
    transforms = Compose([
        Resize(224, 224, interpolation=cv2.INTER_NEAREST),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    # Read image with cv2
    input_img = cv2.imread(img_path)
    
    # Transform image
    input_data = transforms(image=input_img)['image']
    
    # Convert to batch 1 image
    batch_data = torch.unsqueeze(input_data, 0)
    return batch_data

input = preprocess_image("turkish_coffee.jpg").cuda()

**Inference and Postprocess**

In [9]:
# Inference with model
model.eval()
model.cuda()
output = model(input)

In [12]:
def postprocess(output_data):
    # Get class name
    with open('imagenet_classes.txt') as f:
        classes = [line.split(',')[1].strip() for line in f.readlines()]
    
    # Calculate score  
    confidences = torch.nn.functional.softmax(output_data, dim=1)[0] * 100
    
    # Find top predicted classes
    _, indices = torch.sort(output_data, descending=True)
    i = 0
    
    while confidences[indices[0][i]] > 0.5:
        class_idx = indices[0][i]
        
        print(
            "Class:",
            classes[class_idx],
            ", confidence:",
            confidences[class_idx].item(),
            "%, index:",
            class_idx.item()
        )
        
        i += 1
        
postprocess(output)
        


Class: cup , confidence: 94.98471069335938 %, index: 968
Class: espresso , confidence: 3.7401418685913086 %, index: 967
Class: coffee_mug , confidence: 0.6143028140068054 %, index: 504


#### 2. Convert the pytorch model to ONNX format

In [13]:
ONNX_FILE_PATH = 'resnet50.onnx'
torch.onnx.export(
    model,
    input,
    ONNX_FILE_PATH,
    input_names=['input'],
    output_names=['output'],
    export_params=True
)

In [16]:
import onnx

onnx_model = onnx.load(ONNX_FILE_PATH)
onnx.checker.check_model(onnx_model)

#### 3. Visualize model

In [21]:
# Use netron to visualize onnx model
# !netron

: 

#### 4. Initialize model in TensorRT

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt